In [1]:
# example from: https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/
# github: https://github.com/madhug-nadig/Machine-Learning-Algorithms-from-Scratch/blob/master/K%20Means%20Clustering.py

In [2]:
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd
import numpy as np
from math import sqrt
from scipy.interpolate import lagrange
import random
from RandPoly import RandPoly

In [3]:
def generate_functions(features_arr):
    all_functions = []
    for feature in range(len(features_arr)):
        func = RandPoly(name=f"f{feature}", n=1, 
                        R=[(i,x) for i, x in enumerate(list(
            [features_arr[feature],random.randint(2,250)]))]).poly
        all_functions.append(func)
    return all_functions

def generate_shares(func_array, node_id):
    shares = []
    for func in func_array:
        shares.append(func(node_id))
    return shares

def get_feature_distances(arr1, arr2):
    distances = []
    for feature_a, feature_b in zip(arr1, arr2):
        dist = (feature_a-feature_b)**2
        distances.append(dist)
    return distances

def sum_distances(arr):
    return sum(arr)

def reconstruct(shares):
    x = np.arange(1, len(shares) + 1)
    y = shares
    f = lagrange(x, y)
    return f(0)


In [10]:
def euclidean_distance_secure(alice_features, server_features):
    # Check feature length
    assert len(alice_features) == len(server_features), "feature length mismatch!"

    # get private functions
    alice_functions = generate_functions(alice_features)
    server_functions = generate_functions(server_features)

    # get shares
    # f_1 = generate_shares(alice_functions, 1)
    # f_2 = generate_shares(alice_functions, 2)
    # f_3 = generate_shares(alice_functions, 3)
    # g_1 = generate_shares(server_functions, 1)
    # g_2 = generate_shares(server_functions, 2)
    # g_3 = generate_shares(server_functions, 3)

    f_1 = generate_shares(alice_functions, 15645)
    f_2 = generate_shares(alice_functions, 27521)
    f_3 = generate_shares(alice_functions, 37854)
    g_1 = generate_shares(server_functions, 15645)
    g_2 = generate_shares(server_functions, 27521)
    g_3 = generate_shares(server_functions, 37854)

    # send shares to server

    # alice sends alice_shares_2 and alice_shares_3 to server
    # server sends server_shares_1 to alice
     
    # compute distance for each feature
    # alice computes
    distances_1 = get_feature_distances(f_1, g_1)
    # server computes
    distances_2 = get_feature_distances(f_2, g_2)
    distances_3 = get_feature_distances(f_3, g_3)

    
    # get sum of distances
    # alice computes
    s_1 = sum_distances(distances_1)
    # server computes
    s_2 = sum_distances(distances_2)
    s_3 = sum_distances(distances_3)

    #server sends s_2 and s_3 to alice

    # alice computed
    dist_squared = reconstruct([s_1, s_2, s_3])

    distance = np.sqrt(abs(dist_squared))
    
    return distance

In [5]:
# get iris dataset
X_class,y_class = load_iris().data, load_iris().target

# we just want binary classification
X_class = X_class[:100]
y_class = y_class[:100]

X_class,y_class = shuffle(X_class, y_class)

x_df = pd.DataFrame(X_class)
y_df = pd.DataFrame(y_class)

In [31]:
import random

test = random.sample(range(1,7), 3)
test

[4, 6, 5]

In [14]:
a = [0,0]
b = [3,3]

euclidean_distance_secure(a,b)

1857464.6759252248

In [16]:
# Locate the most similar neighbors
def get_neighbors_secure(X_train, y_train, test_row, num_neighbors):
	distances = list()
	for i in range(len(X_train)):
		dist = euclidean_distance_secure(X_train[i], test_row)
		distances.append((X_train[i], dist, y_train[i]))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append([distances[i][0], distances[i][2]])
	return neighbors

In [17]:
pseudo_x_train = [[1,2,3,4,5], [6,7,8,9,10],[90,91,92,93,94], [100,101,102,103,104]]
pseudo_y_train = [0,0,1,1]
pseudo_test_row = [10,11,12,13,14]

get_neighbors_secure(pseudo_x_train, pseudo_y_train, pseudo_test_row, 3)

[[[6, 7, 8, 9, 10], 0], [[1, 2, 3, 4, 5], 0], [[90, 91, 92, 93, 94], 1]]

In [18]:
# Make a classification prediction with neighbors
def predict_classification_secure(X_train, y_train, test_row, num_neighbors):
	neighbors = get_neighbors_secure(X_train, y_train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction

In [19]:
predict_classification_secure(pseudo_x_train, pseudo_y_train, pseudo_test_row, 3)

0

In [21]:
def knn_from_scratch_secure(X_train, X_test, y_train, y_test, num_neighbors):
	y_pred = []
	for test_row in X_test:
		y_pred.append(predict_classification_secure(X_train, y_train, test_row, num_neighbors))

	f1_binary = f1_score(y_test, y_pred, average="binary")
	accuracy = accuracy_score(y_test, y_pred)

	# print("ypred:", y_pred)
	# print("ytest:", y_test)

	return f1_binary, accuracy

In [23]:
#testing

numFolds = 10
num_neighbors = 9
stratifiedKFold = StratifiedKFold(
    n_splits=numFolds, shuffle=True, random_state=86
)

count = 1
avgF1 = 0
avgAcc = 0

X = X_class
y = y_class

for train_index, test_index in stratifiedKFold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    f1, acc = knn_from_scratch_secure(X_train, X_test, y_train, y_test, num_neighbors)
    avgF1 += f1
    avgAcc += acc

    print(f"\tFold {count}:: Average accuracy: {avgAcc}, Average F1 score: {avgF1}")

    count += 1

avgF1 = avgF1/numFolds
avgAcc = avgAcc/numFolds

print(f"Average accuracy: {avgAcc}, Average F1 score: {avgF1}")

	Fold 1:: Average accuracy: 1.0, Average F1 score: 1.0
	Fold 2:: Average accuracy: 2.0, Average F1 score: 2.0
	Fold 3:: Average accuracy: 3.0, Average F1 score: 3.0
	Fold 4:: Average accuracy: 4.0, Average F1 score: 4.0
	Fold 5:: Average accuracy: 5.0, Average F1 score: 5.0
	Fold 6:: Average accuracy: 6.0, Average F1 score: 6.0
	Fold 7:: Average accuracy: 7.0, Average F1 score: 7.0
	Fold 8:: Average accuracy: 8.0, Average F1 score: 8.0
	Fold 9:: Average accuracy: 9.0, Average F1 score: 9.0
	Fold 10:: Average accuracy: 10.0, Average F1 score: 10.0
Average accuracy: 1.0, Average F1 score: 1.0
